# Modeling
Matthew Mandel Hemsley

### Remember: Churn in this dataset represents individuals who left within the last month
Additionally, Accuaracy and Precision will be our most important metrics as we are attempting to minimize Type 2 Errors

In [17]:
#Imports
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.pipeline import Pipeline

In [4]:
churn_df = pd.read_csv('./data/clean_churn.csv')

To begin, we will be constructing a K-Means Clustering model to see if we can identify anything

In [5]:
churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,1,0,1,0,0,1,1,...,1,1,1,1,0,1,1,29.85,29.85,0
1,5575-GNVDE,0,0,0,0,34,1,1,1,2,...,2,1,1,1,1,0,0,56.95,1889.50,0
2,3668-QPYBK,0,0,0,0,2,1,1,1,2,...,1,1,1,1,0,1,0,53.85,108.15,1
3,7795-CFOCW,0,0,0,0,45,0,0,1,2,...,2,2,1,1,1,0,3,42.30,1840.75,0
4,9237-HQITU,1,0,0,0,2,1,1,2,1,...,1,1,1,1,0,1,1,70.70,151.65,1


In [10]:
#Setting index to customerID
churn_df.set_index('customerID',inplace=True)

In [16]:
#Train/Test Split
X = churn_df.drop(columns=['Churn'])
y = churn_df['Churn']

X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=42, stratify=y)

In [18]:
#Pipeline
pipe = Pipeline(
    ('stan', StandardScaler()),
    ('ridge',))

C:\Users\Matthew\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass memory=StandardScaler() as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


ValueError: not enough values to unpack (expected 2, got 1)